## Data collection

### read and filter dataset

In [1]:
# read and filter dataset
df = pd.read_csv('Datasets/Rents & Transactions/rents.csv', sep = ';')

# amounts
feat = 'Contract Amount'
df[feat] = df[feat].map(lambda x: float(str(x).replace(',','.')))
df.loc[df[feat] < 1, feat] = np.nan

feat = 'Annual Amount'
df[feat] = df[feat].map(lambda x: float(str(x).replace(',','.')))
df.loc[df[feat] < 1, feat] = np.nan

df['Contract Years'] = df['Contract Amount'] / df['Annual Amount']
df['Amount'] = df['Annual Amount'] / df['No of Units']

# property size
feat = 'Property Size (sq.m)'
df[feat] = df[feat].map(lambda x: float(str(x).replace(',','.')))

# dates feats
df['Transaction Date'] = pd.to_datetime(df['Registration Date'], errors = 'coerce', dayfirst = True)
df['dt_month'] = pd.to_datetime(df['Transaction Date'].astype(str).str[:7])
df['dt_year'] = df['Transaction Date'].astype(str).str[:4].astype(int)

df['tr_year_num'] = df['dt_year']
df['tr_month_num'] = df['dt_month'].dt.month

# invalid property id - replace as contract number (with minus)
f1 = df['Property ID'] == -3
df.loc[f1, 'Property ID'] = -df.loc[f1, 'Ejari Contract Number']

# filtering
# df[df.duplicated(subset = ['Transaction Number'])]
df.drop_duplicates(keep = 'first', inplace = True)

f1 = df['Transaction Date'] > '2012-01-01'
f2 = ~df['Amount'].isnull()
filt = f1&f2
print(df.shape)
df = df[filt].sort_values('Transaction Date').reset_index(drop = True)
print(df.shape)

(5471932, 29)
(5417110, 29)


In [2]:
# last property id value
feat = 'Property ID'
df[f'{feat}_last'] = df.groupby([feat]).Amount.shift(1)

### cat to num or flags

In [3]:
# flags
feat = 'Usage'
repl1 = {'Residential': 1}
df[feat] = df[feat].map(repl1).fillna(0).astype(int)

feat = 'Version'
repl1 = {'New': 1}
df[feat] = df[feat].map(repl1).fillna(0).astype(int)

feat = 'Is Free Hold?'
repl1 = {'Free Hold': 1}
df[feat] = df[feat].map(repl1).fillna(0).astype(int)

feat = 'Parking'
df[feat] = df[feat].notnull().astype(int)

### cats cleaning - unpopular as null

In [4]:
cnt_min = 99

In [5]:
# Property Type
feat = 'Property Type'
repl1 = {
    'Villa': 'Building',
    'Virtual Unit': 'Virtual_Unit',
}
df[feat] = df[feat].fillna('').str.strip().map(lambda x: repl1[x] if x in repl1 else x)

df[feat].value_counts()

Property Type
Unit            4911427
Building         421986
Virtual_Unit      47517
Land              36180
Name: count, dtype: int64

In [6]:
# Property Sub Type
feat = 'Property Sub Type'
df[feat] = df['Property Type'] + '_' + df[feat].fillna('')

repl1 = {}
df[feat] = df[feat].fillna('').str.strip().map(lambda x: repl1[x] if x in repl1 else x)

repl1 = df[feat].value_counts()
repl1 = repl1[repl1 > cnt_min]
df[feat] = df[feat].map(lambda x: x if x in repl1.index else '')
df[feat].value_counts()

Property Sub Type
Unit_Flat                              3120092
Unit_Shop                               588553
Unit_Office                             551004
Unit_Labor Camps                        380951
Building_Villa                          371420
Unit_Studio                              75959
Unit_Warehouse                           68249
Virtual_Unit_Office                      43574
Building_Complex Villas                  41379
Land_                                    36179
Unit_Hotel                               31901
Unit_Showroom                            25602
Unit_Hotel apartments                    14818
Unit_Portacabin                           8340
Unit_Restaurant                           5724
Unit_Store                                4770
Unit_Building                             3677
Unit_Warehouse complex                    3661
Virtual_Unit_Shop                         3124
Unit_Parking                              2916
Unit_Mezzanine                            

In [7]:
# Nearest Landmark
feat = 'Nearest Landmark'
repl1 = {}
df[feat] = df[feat].fillna('').str.strip().map(lambda x: repl1[x] if x in repl1 else x)

repl1 = df[feat].value_counts()
repl1 = repl1[repl1 > cnt_min]
df[feat] = df[feat].map(lambda x: x if x in repl1.index else '')
df[feat].value_counts()

Nearest Landmark
Dubai International Airport          2161924
Burj Khalifa                          640846
Burj Al Arab                          639603
Sports City Swimming Academy          480099
Downtown Dubai                        383386
                                      344937
Expo 2020 Site                        326869
IMG World Adventures                  187378
Motor City                            152170
Dubai Cycling Course                   38853
Al Makhtoum International Airport      35909
Dubai Parks and Resorts                13706
Global Village                          9633
Hamdan Sports Complex                   1620
Jabel Ali                                177
Name: count, dtype: int64

In [8]:
# Nearest Mall
feat = 'Nearest Mall'
repl1 = {}
df[feat] = df[feat].fillna('').str.strip().map(lambda x: repl1[x] if x in repl1 else x)

repl1 = df[feat].value_counts()
repl1 = repl1[repl1 > cnt_min]
df[feat] = df[feat].map(lambda x: x if x in repl1.index else '')
df[feat].value_counts()

Nearest Mall
Dubai Mall              1974789
City Centre Mirdif      1303629
                         597315
Mall of the Emirates     572519
Marina Mall              551957
Ibn-e-Battuta Mall       416901
Name: count, dtype: int64

In [9]:
# Nearest Metro
feat = 'Nearest Metro'
repl1 = {
    'Jumeirah Beach Resdency': 'Jumeirah Beach Residency'
}
df[feat] = df[feat].fillna('').str.strip().map(lambda x: repl1[x] if x in repl1 else x)

repl1 = df[feat].value_counts()
repl1 = repl1[repl1 > cnt_min]
df[feat] = df[feat].map(lambda x: x if x in repl1.index else '')
df[feat].value_counts()

Nearest Metro
                                        560777
Rashidiya Metro Station                 508638
Noor Bank Metro Station                 269212
Baniyas Square Metro Station            223184
Al Fahidi Metro Station                 184519
Dubai Internet City                     173653
Business Bay Metro Station              163018
STADIUM Metro Station                   155916
DANUBE Metro Station                    155221
Palm Deira Metro Stations               154577
Sharaf Dg Metro Station                 153011
ADCB Metro Station                      144405
Airport Free Zone                       136114
Buj Khalifa Dubai Mall Metro Station    127575
Harbour Tower                           125352
Al Nahda Metro Station                  122083
Salah Al Din Metro Station              121814
Etisalat Metro Station                  112324
Nakheel Metro Station                   108244
Burjuman Metro Station                  101000
Damac Properties                         98197

In [10]:
# Project
feat = 'Project'
print(df[feat].nunique())
df[feat] = df[feat].str.lower().str.strip()
print(df[feat].nunique())

repl1 = df[feat].value_counts()
repl1 = repl1[repl1 > cnt_min]
df[feat] = df[feat].map(lambda x: x if x in repl1.index else '')
print(df[feat].nunique())
df[feat].value_counts()

1100
1100
844


Project
                             4633707
remraam                        20419
remraam - al ramth 2           20353
remraam - al ramth             20353
sky courts                     16059
                              ...   
aces chateau                     102
modelux tower 1                  102
micasa avenue                    101
damac hills - the flora          101
the pulse residence plaza        100
Name: count, Length: 844, dtype: int64

In [11]:
# Project
feat = 'Area'
print(df[feat].nunique())
df[feat] = df[feat].str.lower().str.strip()
print(df[feat].nunique())

repl1 = df[feat].value_counts()
repl1 = repl1[repl1 > cnt_min]
df[feat] = df[feat].map(lambda x: x if x in repl1.index else '')
print(df[feat].nunique())
df[feat].value_counts()

216
216
185


Area
al warsan first                      242724
jabal ali first                      170135
al karama                            167738
naif                                 165476
marsa dubai                          159018
business bay                         139731
al nahda second                      133825
al suq al kabeer                     126926
jabal ali industrial first           121841
al goze industrial second            120856
al mararr                            120191
mirdif                               117737
al barsha first                      117194
nadd hessa                           117147
al murqabat                          113648
al thanyah fifth                     109789
al qusais first                      101631
mankhool                             101460
dubai investment park first           89584
muhaisanah fourth                     87399
dubai investment park second          84676
al raffa                              83048
al goze third              

### cats encoding

In [12]:
# categorical feats encoding
feats_cat = [    
    'Property Type',
    'Property Sub Type',
    'Area',
    'Nearest Metro',
    'Nearest Mall',
    'Nearest Landmark',
    'Project',
]

# label encoder
for feat in feats_cat:
    lbl = preprocessing.LabelEncoder()
    df[f'{feat}_lbl'] = lbl.fit_transform(df[feat])
    

# one-hot maybe..
# lbl = preprocessing.OneHotEncoder(sparse_output=False, drop='first')
# df = pd.concat([df, pd.DataFrame(lbl.fit_transform(df[[f+'_lbl']]), columns = lbl.get_feature_names_out())], axis = 1)
# del df[f+'_lbl']

In [13]:
# target encoders by average value in the previous month
for feat in feats_cat[1:]:
    feats_gr = ['dt_month', 'Property Type', feat]
    df1 = df.groupby(feats_gr).Amount.mean().reset_index()
    df1[f'{feat}_trg'] = df1.groupby(feats_gr[1:]).Amount.shift(1)
    del df1['Amount']    
    df = df.merge(df1, how = 'left')

In [14]:
df[df['Transaction Date'] < '2022-06-01'].to_pickle('data2.pkl')
print(df.shape)
df[:5]

(5417110, 43)


,Ejari Contract Number,Registration Date,Start Date,End Date,Property ID,Version,Area,Contract Amount,Annual Amount,Is Free Hold?,Property Size (sq.m),Property Type,Property Sub Type,Number of Rooms,Usage,Nearest Metro,Nearest Mall,Nearest Landmark,Parking,No of Units,Master Project,Project,Contract Years,Amount,Transaction Date,dt_month,dt_year,tr_year_num,tr_month_num,Property ID_last,Property Type_lbl,Property Sub Type_lbl,Area_lbl,Nearest Metro_lbl,Nearest Mall_lbl,Nearest Landmark_lbl,Project_lbl,Property Sub Type_trg,Area_trg,Nearest Metro_trg,Nearest Mall_trg,Nearest Landmark_trg,Project_trg
0,56129335,01/01/2012 10:31:05,10/12/2011 00:00:00,09/12/2012 00:00:00,311295,1,eyal nasser,"65,000.00","65,000.00",0,15.05,Unit,Unit_Shop,NaN,0,Palm Deira Metro Stations,Dubai Mall,Dubai International Airport,0,1,NaN,,1.00,"65,000.00",2012-01-01 10:31:05,2012-01-01,2012,2012,1,NaN,2,47,110,46,2,6,0,NaN,NaN,NaN,NaN,NaN,NaN
1,85678259,01/01/2012 10:49:45,01/01/2012 00:00:00,31/12/2012 00:00:00,301083,1,al sabkha,"95,700.00","95,700.00",0,42.36,Unit,Unit_Shop,NaN,0,Palm Deira Metro Stations,Dubai Mall,Dubai International Airport,0,1,NaN,,1.00,"95,700.00",2012-01-01 10:49:45,2012-01-01,2012,2012,1,NaN,2,47,73,46,2,6,0,NaN,NaN,NaN,NaN,NaN,NaN
2,81975881,01/01/2012 11:17:47,26/12/2011 00:00:00,25/12/2012 00:00:00,284859,1,al mararr,"28,000.00","28,000.00",0,36.79,Unit,Unit_Flat,NaN,0,Palm Deira Metro Stations,Dubai Mall,Dubai International Airport,0,1,NaN,,1.00,"28,000.00",2012-01-01 11:17:47,2012-01-01,2012,2012,1,NaN,2,29,49,46,2,6,0,NaN,NaN,NaN,NaN,NaN,NaN
3,88277598,01/01/2012 11:47:11,01/01/2012 00:00:00,31/12/2012 00:00:00,354438,1,al mararr,"97,000.00","97,000.00",0,39.02,Unit,Unit_Shop,NaN,0,Palm Deira Metro Stations,Dubai Mall,Dubai International Airport,0,1,NaN,,1.00,"97,000.00",2012-01-01 11:47:11,2012-01-01,2012,2012,1,NaN,2,47,49,46,2,6,0,NaN,NaN,NaN,NaN,NaN,NaN
4,64535887,01/01/2012 12:17:50,01/01/2012 00:00:00,31/12/2012 00:00:00,232470,1,al mararr,"26,000.00","26,000.00",0,25.46,Unit,Unit_Flat,NaN,0,Palm Deira Metro Stations,Dubai Mall,Dubai International Airport,0,1,NaN,,1.00,"26,000.00",2012-01-01 12:17:50,2012-01-01,2012,2012,1,NaN,2,29,49,46,2,6,0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
list(df.columns)

['Ejari Contract Number',
 'Registration Date',
 'Start Date',
 'End Date',
 'Property ID',
 'Version',
 'Area',
 'Contract Amount',
 'Annual Amount',
 'Is Free Hold?',
 'Property Size (sq.m)',
 'Property Type',
 'Property Sub Type',
 'Number of Rooms',
 'Usage',
 'Nearest Metro',
 'Nearest Mall',
 'Nearest Landmark',
 'Parking',
 'No of Units',
 'Master Project',
 'Project',
 'Contract Years',
 'Amount',
 'Transaction Date',
 'dt_month',
 'dt_year',
 'tr_year_num',
 'tr_month_num',
 'Property ID_last',
 'Property Type_lbl',
 'Property Sub Type_lbl',
 'Area_lbl',
 'Nearest Metro_lbl',
 'Nearest Mall_lbl',
 'Nearest Landmark_lbl',
 'Project_lbl',
 'Property Sub Type_trg',
 'Area_trg',
 'Nearest Metro_trg',
 'Nearest Mall_trg',
 'Nearest Landmark_trg',
 'Project_trg']